<div style="font-size: 18pt;">- RMP dataset</div><br/>
<div style="font-size: 18pt;">- emotion classification</div><br/>
<div style="font-size: 18pt;">- zero-shot classification</div><br/>

In [1]:
import torch

In [2]:
#exetute this cell only in Google Colab
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 46.8 MB/s 
     |████████████████████████████████| 86 kB 3.3 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import pipeline
import os
import torch
import numpy as np
import pandas as pd
import pickle

In [4]:
#exetute this cell only in Google Colab
from google.colab import files
import io

In [8]:
#exetute this cell only in Google Colab
uploaded = files.upload()
rmp = pd.read_csv(io.BytesIO(uploaded['rmp_small.csv']))

Saving rmp_small.csv to rmp_small.csv


In [ ]:
rmp = pd.read_csv("data/rmp_small.csv") #for local drive

In [9]:
rmp

,review,score
0,"he can be a mean man sometimes, but just get o...",2
1,"He's fun, but he speeds through lectures and l...",2
2,Dr. Foster is an amazing teacher. One of my fa...,4
3,She is one of the greatest teacher I have ever...,4
4,Professor Bird was very easy and relaxed. If y...,3
...,...,...
97540,Charismatic and enthusiastic but that's it . H...,0
97541,"Great books - Dickens, Eliot, etc. A little d...",3
97542,Awesome professor. Doesnt stop explaining unti...,4
97543,"He has a really high standard, but grades fair...",4


In [10]:
sentences = list(rmp['review'])
labels = list(rmp['score'])

In [11]:
#remove nan from both lists
selnan = list()
for idx,name in enumerate(sentences):
    if name == '' or pd.isnull(name):
        selnan.append(idx)
sentences = [sentences[i] for i, e in enumerate(sentences) if i not in selnan]
labels = [labels[i] for i, e in enumerate(labels) if i not in selnan]
(len(sentences), len(labels))

(97545, 97545)

In [12]:
classifier = pipeline("zero-shot-classification", 
                      model = "valhalla/distilbart-mnli-12-3",
                      device = 0
)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/977M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [14]:
l1 = "enthusiastic student with high expectations"
l2 = "student with exceeded expectations"
l3 = "disappointed student with unmet expectations"
l4 = "indifferent or uninterested student with no expectations"
l5 = "other"
candidate_labels=[l1, l2, l3, l4, l5]

In [15]:
#predition = classifier(sentences, candidate_labels=candidate_labels)

In [ ]:
predition

In [75]:
l_max_labels = [val['labels'][0] for val in predition]

In [81]:
df = pd.DataFrame(list(zip(l_max_labels, labels)), columns =['student_expectation', 'score'])
df

,student_expectation,score
0,other,2
1,enthusiastic student with high expectations,2
2,student with exceeded expectations,4
3,student with exceeded expectations,4
4,student with exceeded expectations,3
...,...,...
97540,other,0
97541,other,3
97542,student with exceeded expectations,4
97543,student with exceeded expectations,4


In [82]:
#calculate crosstabulations
result = pd.crosstab(df["student_expectation"], df["score"])
result

score,0,1,2,3,4
student_expectation,,,,,
disappointed student with unmet expectations,4219,2310,1426,971,672
enthusiastic student with high expectations,130,264,756,2340,3424
indifferent or uninterested student with no expectations,754,502,396,395,344
other,10635,6864,7979,11824,12605
student with exceeded expectations,253,430,1652,7820,18580
